In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [1]:
import tensorflow as tf
from sklearn import cross_validation
from sklearn import metrics as mt
from sklearn import utils as ut
import gc
import csv
import numpy as np
import os.path as path
from os import listdir
from PIL import Image

In [2]:
with open('./photo_to_levels.csv') as f:
    food_to_label = []
    for row in csv.DictReader(f, skipinitialspace=True):
        element = {}
        for k, v in row.items():
            if k == "id":
                element['id'] = str(v) + ".jpg"
            elif k == "levels":
                labels_raw = np.array(str(v).split(' '))
                labels = [0] * 9
                labels_int = []
                try:
                    for lb in labels_raw:
                        labels[int(str(lb))] = 1
                        labels_int.append(int(lb))
                except ValueError:
                    print "Failure with value", lb, "labels lenght", len(labels_raw), "content:", v
                element['labels'] = labels
                element['labels_raw'] = labels_int
            else :
                print "No idea what you just passed!"
        
        if len(element['labels_raw']) is not 0:
            food_to_label.append(element)
        else:
            print "Picture", element['id'], "has no labels and is being ignored!"

if len(set([element['id'] for element in food_to_label])) != len(food_to_label):
    print('something\'s wrong!')

Failure with value None labels lenght 1 content: None
Picture 329580.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 253479.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 142477.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 64965.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 324888.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 206582.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 20435.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 346344.jpg has no labels and is being ignored!
Failure with value None labels lenght 1 content: None
Picture 80447.jpg has no labels and is being ignored!
Failure with value Non

In [3]:
proportions = []
for lb in range(9):
    l = len([element for element in food_to_label if lb in element['labels_raw']])/float(len(food_to_label))
    print "Label", lb, "is present at", int(l*100), "% with respect to all other labels"
    proportions.append(l)

Label 0 is present at 22 % with respect to all other labels
Label 1 is present at 59 % with respect to all other labels
Label 2 is present at 64 % with respect to all other labels
Label 3 is present at 48 % with respect to all other labels
Label 4 is present at 42 % with respect to all other labels
Label 5 is present at 73 % with respect to all other labels
Label 6 is present at 77 % with respect to all other labels
Label 7 is present at 37 % with respect to all other labels
Label 8 is present at 50 % with respect to all other labels


In [4]:
# Data dir
imagesDir = './data/SampleFoodClassifier_Norm_100'

# Filter out images which might not be present in the folder but are present in the csv file
files = [f for f in listdir(imagesDir) if path.isfile(path.join(imagesDir, f))]
food_to_label = [element for element in food_to_label if element['id'] in files]

del files[:]
gc.collect()

print "The new length of the data is", len(food_to_label)

# Parameters
test_size = 500
learning_rate_start= .001
training_size = 50
training_iters = 100
dropout = 0.75 # Dropout, probability to keep units

# Network Parameters
# !! Images: 100x100 RGB = 100, 100, 3
w, h, channels = [100, 100, 3]
n_classes = 9
print "Width, Height and channels:", w, h, channels, ". Number of classes:", n_classes

# tf Graph input
x = tf.placeholder(tf.float32, [None, w, h, channels])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

The new length of the data is 1297
Width, Height and channels: 100 100 3 . Number of classes: 9


In [5]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print "PLEASE MODIFY WD1 TO", conv2.get_shape().as_list()[1], "*",conv2.get_shape().as_list()[2], "*64"

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [6]:
# Store layers weight & bias
sdev= 0.01
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.truncated_normal([5, 5, channels, 32], stddev=sdev)),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=sdev)),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.truncated_normal([25*25*64, 1024], stddev=sdev)),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.truncated_normal([1024, n_classes], stddev=sdev))
}

biases = {
    'bc1': tf.Variable(tf.truncated_normal([32], stddev=sdev)),
    'bc2': tf.Variable(tf.truncated_normal([64], stddev=sdev)),
    'bd1': tf.Variable(tf.truncated_normal([1024], stddev=sdev)),
    'out': tf.Variable(tf.truncated_normal([n_classes], stddev=sdev))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits (pred, y))

# optimizer without adapted learning_rate
#optimizer = tf.train.AdamOptimizerOptimizer(learning_rate=learning_rate).minimize(cost)

#optimizer with adapted learning_rate
step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(learning_rate_start, step, 1, 0.9999)

optimizer = tf.train.AdamOptimizer(rate).minimize(cost, global_step=step)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
y_p = tf.argmax(pred, 1)


# Gives an array of arrays, where each position represents % of belonging to respective classs. Eg: a[0.34, 0.66] --> class 0 : 34%, class 1: 66%
classes = tf.nn.softmax(pred)


def label_class(x):
    for i in range(0,len(x)):
        print i, ":", x[i]

# Initializing the variables
init = tf.initialize_all_variables()

PLEASE MODIFY WD1 TO 25 * 25 *64


In [7]:
def dataSplit(array, size):
    split = [element['id'] for element in ut.shuffle(array, n_samples=size, random_state=37)]
    return [element for element in array if element['id'] in split], [element for element in array if element['id'] not in split]

def dataAndLabels(array):
    return [np.array(Image.open(path.join(imagesDir, element['id']))) for element in array], [element['labels'] for element in array]

In [8]:
# Get out a sample of data for testing
## IMPORTANT: RUN ONLY ONCE!
test, food_to_label = dataSplit(food_to_label, test_size)

In [9]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Keep training until reach max iterations
    for epoch in range(training_iters):
        if len(food_to_label) < training_size:
            del ix[:]
            del iy[:]
            del batch[:]
            break
        # Fit training using batch data
        print "Loading batch...",
        batch, food_to_label = dataSplit(food_to_label, training_size)
        ix, iy = dataAndLabels(batch)
        print "bactch loaded!"
        
        print "Running optimizer...",
        sess.run(optimizer, feed_dict={x: ix, y: iy, keep_prob: 1.})
        print "done!"
        # Compute average loss
        loss, acc = sess.run([cost, accuracy], feed_dict={x: ix, y: iy, keep_prob: 1.})
        # Display logs per epoch step
        print "Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        del ix[:]
        del iy[:]
        del batch[:]
    print "Optimization Finished!"

    runs = 0
    acc = 0.
    y_pred = []
    y_test = []
    class_pred = []
    for i in range(0, test_size, 30):
        if i+30 < test_size:
            x_test, y_test_partial = dataAndLabels(test[i:i+30])
        else:
            x_test, y_test_partial = dataAndLabels(test[i:])
        val_accuracy, y_pred_i, cls = sess.run([accuracy, y_p, classes], feed_dict={x: x_test, y: y_test_partial, keep_prob: 1.})
        acc += val_accuracy
        y_test.extend(y_test_partial)
        y_pred.extend(y_pred_i)
        class_pred.extend(cls)
        runs += 1
        print "Partial testing accuracy:", acc/runs
    
    #metrics
    print "Validation accuracy:", acc/runs
    y_true = np.argmax(y_test,1)
    print "Precision for each class:"
    label_class(mt.precision_score(y_true, y_pred, average=None))
    print "Recall for each class:"
    label_class(mt.recall_score(y_true, y_pred, average=None))
    print "F1_score for each class:"
    label_class(mt.f1_score(y_true, y_pred, average=None))
    print "confusion_matrix"
    print mt.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = mt.roc_curve(y_true, y_pred)

Loading batch... bactch loaded!
Running optimizer... done!
Iter 0, Minibatch Loss= 9.143058, Training Accuracy= 0.04000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 1, Minibatch Loss= 1.459610, Training Accuracy= 0.00000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 2, Minibatch Loss= 0.744047, Training Accuracy= 0.00000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 3, Minibatch Loss= 0.671756, Training Accuracy= 0.02000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 4, Minibatch Loss= 0.843290, Training Accuracy= 0.08000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 5, Minibatch Loss= 0.660541, Training Accuracy= 0.04000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 6, Minibatch Loss= 0.678093, Training Accuracy= 0.02000
Loading batch... bactch loaded!
Running optimizer... done!
Iter 7, Minibatch Loss= 0.693406, Training Accuracy= 0.00000
Loading batch... bactch loaded!
Running 

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


ValueError: Data is not binary and pos_label is not specified

In [10]:
for i in range(len(y_test)):
    print "For", i, "as", y_test[i]
    for j in range(9):
        print "\t", j, "@", class_pred[i][j]*100
    print "\n"

For 0 as [1, 1, 0, 1, 0, 1, 1, 0, 1]
	0 @ 9.82696712017
	1 @ 11.2135775387
	2 @ 11.1326798797
	3 @ 9.78526175022
	4 @ 10.7543088496
	5 @ 12.3724676669
	6 @ 12.5287219882
	7 @ 12.036306411
	8 @ 10.3497050703


For 1 as [0, 1, 1, 1, 1, 1, 1, 1, 0]
	0 @ 9.9776186049
	1 @ 11.2694010139
	2 @ 11.1507683992
	3 @ 9.95276793838
	4 @ 10.8356311917
	5 @ 12.1411085129
	6 @ 12.3479813337
	7 @ 11.8496306241
	8 @ 10.4750908911


For 2 as [0, 1, 1, 0, 1, 1, 1, 1, 1]
	0 @ 9.85997021198
	1 @ 11.2238332629
	2 @ 11.1238799989
	3 @ 9.774235636
	4 @ 10.7442863286
	5 @ 12.3403124511
	6 @ 12.5204712152
	7 @ 11.9863368571
	8 @ 10.4266710579


For 3 as [0, 0, 1, 1, 1, 1, 1, 1, 1]
	0 @ 9.99502316117
	1 @ 11.255581677
	2 @ 11.1516535282
	3 @ 9.97147187591
	4 @ 10.8291111887
	5 @ 12.1333047748
	6 @ 12.3336464167
	7 @ 11.8240848184
	8 @ 10.5061262846


For 4 as [1, 1, 0, 1, 0, 0, 0, 0, 1]
	0 @ 9.99210327864
	1 @ 11.2431056798
	2 @ 11.1563779414
	3 @ 9.95618253946
	4 @ 10.828191787
	5 @ 12.1439129114
	6 @ 12.3412959